## Magic Functions
https://blog.csdn.net/zhangke0426/article/details/122929667

python 定义类时中，以双下划线开头，以双下划线结尾函数为魔法函数

- 魔法函数可以定义类的特性
- 魔法函数是解释器提供的功能
- 魔法函数只能使用 python 提供的魔法函数，不能自定义

常用的包括： __ init__()、__ str__()、__ new__()、__ unicode__()、 __ call__()、 __ len__()、 __repr__()、__ setattr__()、 __ getattr__()、 __ getattribute__()、 __ delattr__()、__ setitem__()、 __ getitem__()、__ delitem__()、 __ iter__()、__ del__()、 __dir__()、__dict__()、__exit__()，__enter(), __all__()等函数。

### 非数学运算类魔法函数

#### __str__ 和 __repr__
均是用于显示的，__str__ 用于 print，__repr__ 用于直接显示

In [1]:
# __str__ 和 __repr__
class A:
    def __str__(self):
        return 'str'
    def __repr__(self):
        return 'repr'
a = A()
print(a) # 使用print时，调用__str__函数
a # 不使用print时，调用__repr__函数，如果没有__repr__函数，则调用__str__函数，同理，如果没有__str__函数，则调用__repr__函数

str


repr

#### 集合、序列相关：__len__函数、__getitem__函数、__setitem__函数、__delitem__函数和__contains__函数

In [7]:
# 构建一个类，展示上述函数的使用
class A:
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        return self.data[index]
    def __setitem__(self, index, value):
        self.data[index] = value
    def __delitem__(self, index):
        del self.data[index]
    def __contains__(self, value):
        return value in self.data
a = A([1, 2, 3, 4, 5])
print(len(a)) # 调用__len__函数
print(a[0]) # 调用__getitem__函数
a[0] = 10 # 调用__setitem__函数
print(a[0]) # 调用__getitem__函数
del a[0] # 调用__delitem__函数
print(a[0]) # 调用__getitem__函数
print(10 in a) # 调用__contains__函数

5
1
10
2
False


#### __call__函数
该方法的功能类似于在类中重载 () 运算符，使得类实例对象可以像调用普通函数那样，以“对象名()”的形式使用。作用：为了将类的实例对象变为可调用对象。

In [8]:
# __call__函数
class A:
    def __call__(self, *args, **kwargs):
        print('call')
a = A()
a() # 调用__call__函数

call


#### __exit__ 和 __enter__ 函数
__exit__和__enter__函数是与with语句的组合应用的，用于上下文管理。

1. __enter(self)__：负责返回一个值，该返回值将赋值给as子句后面的var_name，通常返回对象自己，即“self”。函数优先于with后面的“代码块”(statements1,statements2,……)被执行。

2. __exit__(self, exc_type, exc_val, exc_tb)：负责执行“清理”工作，比如释放资源等。函数在with后面的“代码块”(statements1,statements2,……)执行完毕后被调用，即“代码块”执行完毕后，执行__exit__函数。

In [10]:
# __enter__函数
class A:
    def __enter__(self):
        print('enter中的逻辑')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit中的逻辑')

with A() as a:
    print('进入with的逻辑')

enter中的逻辑
进入with的逻辑
exit中的逻辑


一个常见的用途是在pytorch中使用with语句，如下所示：
```python
import torch
with torch.no_grad():
    # 不进行梯度更新
    pass
```
而在改类中实际上是使用了__enter__和__exit__函数。
```python
import torch
def __enter__(self) -> None:
    self.prev = torch.is_grad_enabled()
    torch.set_grad_enabled(False)
def __exit__(self, exc_type: Any, exc_value: Any, traceback: Any) -> None:
    torch.set_grad_enabled(self.prev)
```


#### __new__函数与__init__函数
1. __new__函数：用于创建对象，是 **类（静态）** 方法，返回一个实例对象。该方法在__init__方法之前调用，用于创建实例对象。__new__方法的第一个参数是cls，表示要实例化的类，其余参数将会传递给__init__方法。
2. __init__函数：用于初始化对象，是实例方法，不返回任何内容。该方法在__new__方法之后调用，用于初始化实例对象。__init__方法的第一个参数是self，表示实例对象本身，其余参数将会传递给__new__方法。
3. __new__函数的返回值是一个实例对象，而__init__函数没有返回值。Python中真正的构造方法是__new__ 方法，__init__方法只是用来将传入的参数初始化到实例对象中。

In [1]:
# python中使用__new__函数实现单例模式
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        print('__new__')
        if not cls._instance:
            print('create instance')
            cls._instance = super(Singleton, cls).__new__(cls)
        return cls._instance
    def __init__(self, name):
        print('__init__')
        self.name = name
a = Singleton('a')
b = Singleton('c')
print(a)
print(b)
print(b.name)

__new__
create instance
__init__
__new__
__init__
c


4. 一些关于__new__的应用:TODO

#### __getattr__、__setattr__、__delattr__函数
1. 当我们访问一个不存在的属性的时候，会抛出异常，提示我们不存在这个属性。而这个异常就是__getattr__方法抛出的
2. __setattr__方法用于设置属性值，当我们设置属性值的时候，会调用这个方法
3. __delattr__方法用于删除属性值，当我们删除属性值的时候，会调用这个方法

In [11]:
# __getattr__、__setattr__、__delattr__函数
class A:
    def __init__(self):
        self.data = {'a': 123}
    def __getattr__(self, name):
        print('getattr')
        return self.data[name] #  如果不存在这个属性，会抛出异常
    def __setattr__(self, name, value):
        print('setattr')
        self.__dict__[name] = value
    def __delattr__(self, name):
        print('delattr')
        del self.__dict__[name]
a = A()
print(a.__dict__)
print(a.a) # 调用__getattr__函数
a.b = 123 # 调用__setattr__函数
print(a.__dict__)
del a.b # 调用__delattr__函数
print(a.__dict__)

setattr
{'data': {'a': 123}}
getattr
123
setattr
{'data': {'a': 123}, 'b': 123}
delattr
{'data': {'a': 123}}


#### __getattribute__，__setattr__

1. __getattribute__：该方法在访问属性时自动调用，无论属性是否存在，都会调用该方法。该方法的优先级高于__getattr__方法。如果类中定义了__getattribute__方法，那么在访问属性时，就会调用__getattribute__方法，而不会调用__getattr__方法。__getattribute__是属性访问拦截器，就是当这个类的属性被访问时，会自动调用类的__getattribute__方法。
2. __setattr__：该方法在设置属性时自动调用，无论属性是否存在，都会调用该方法。该方法的优先级高于__setattr__方法。如果类中定义了__setattr__方法，那么在设置属性时，就会调用__setattr__方法，而不会调用__setattr__方法。__setattr__是属性设置拦截器，就是当这个类的属性被设置时，会自动调用类的__setattr__方法。
3. __getattribute__和__setattr__方法的优先级高于__getattr__和__setattr__方法。

#### __dir__函数

1. dir() 函数，通过此函数可以某个对象拥有的所有的属性名和方法名，该函数会返回一个包含有所有属性名和方法名的有序列表。

### 数学运算类魔法函数
1. 类似于运算符重载，可以自定义类的运算符
2. 包括__ add__、__sub__、__mul__、__truediv__、__floordiv__、__mod__、__pow__、__and__、__or__、__xor__、__lshift__、__rshift__、__neg__、__pos__、__abs__、__invert__、__iadd__、__isub__、__imul__、__itruediv__、__ifloordiv__、__imod__、__ipow__、__iand__、__ior__、__ixor__、__ilshift__、__irshift__、__complex__、__int__、__float__、__round__、__index__等函数。